# Analysis of Land Registry data on titles owned by individuals with an overseas correspondence address

This notebook analyses data on the number of Land Registry titles in England & Wales owned by individuals with an overseas correspondence address between 2010 and 2021.

This data is interesting beacuse it's a proxy (see below) for overseas citizens buying second homes or buying investment property in England & Wales, which otherwise we don't have any statistics on.

**Data source**

This data was obtained via Freedom of Information requests:
- data for [Jan 2010, 2012 and 2014](https://www.whatdotheyknow.com/alaveteli_pro/info_requests/titles_owned_by_overseas_individ_3)
- data for [Jan 2016, 2018 and 2020](https://www.whatdotheyknow.com/alaveteli_pro/info_requests/titles_owned_by_overseas_individ_2)
- data for [Aug 2021](https://www.whatdotheyknow.com/alaveteli_pro/info_requests/titles_owned_by_overseas_individ_4).

**Technical notes on the data**

The data is titles owned by individuals "with an overseas correspondence address". The Land Registry [asks for a correspondence address](https://www.gov.uk/government/publications/registered-titles-whole-transfer-tr1/guidance-completing-form-tr1-for-the-transfer-of-registered-property#panel-6-address) when a property is registered, so it can write to that address if an attempt is made to change the entry on the register.

Normally people just use the address of the property - so an overseas correspondence address is a good indication that they don't mainly live at the property, either because it's a second home, or because they rent it. (Edge case: it's possible that a UK resident citizen might also register an overseas correspondence address, but this is likely to be a very small number of people.)

And some other important points:

- These are titles owned by individuals, not companies. Data on titles owned by overseas companies is published separately.
- Titles aren't necessarily residential property: they can also be commercial properties or land.
- One title can also contain multiple properties, though this is rare.
- The data shows stock, not flow, i.e. it is the total number of properties owned at each point in time, not sales. 

In [4]:
import glob
import pandas as pd

## Headline findings

In [5]:
df_by_country = pd.read_csv("by_country.csv")
df_by_country.sum()

COUNTRY                 HONG KONGJERSEYSINGAPOREUNITED STATES OF AMERI...
TITLES_2010_01                                                      87810
TITLES_2012_01                                                     120042
TITLES_2014_01                                                     152303
TITLES_2016_01                                                     184000
TITLES_2018_01                                                     215569
TITLES_2020_01                                                     243645
TITLES_2021_08                                                     247016
abs_change_10_21                                                   159206
percent_change_10_21                                                  inf
dtype: object

Change from year to year. Some important tax events during this period:

1. In 2013 the Government introduced an [annual tax on UK resi property owned by a company](https://www.gov.uk/guidance/annual-tax-on-enveloped-dwellings-the-basics) worth over £2 million, and over the following years it reduced the threshold for this tax. This will have increased the incentives for owning as an individual rather than via a company.
2. In April 2017, the Government [brought all UK residential property within the scope of inheritance tax](https://www.cbw.co.uk/2021/05/reforms-to-the-taxation-of-non-doms-how-have-the-changes-introduced-in-2017-affected-non-doms-living-in-the-uk-today/), again reducing the incentives to own via an overseas company - I think.
2. In April 2021, the Government introduced a 2% surcharge on stamp duty for property acquired by overseas tax residents, though it's probably too early to see any effect of this yet.

In [10]:
print(df_by_country.TITLES_2012_01.sum() - df_by_country.TITLES_2010_01.sum())
print(df_by_country.TITLES_2014_01.sum() - df_by_country.TITLES_2012_01.sum())
print(df_by_country.TITLES_2016_01.sum() - df_by_country.TITLES_2014_01.sum())
print(df_by_country.TITLES_2018_01.sum() - df_by_country.TITLES_2016_01.sum())
# A slight slowing in the rise between 2018 and 2020, perhaps the effect of tax changes in 2017. 
print(df_by_country.TITLES_2020_01.sum() - df_by_country.TITLES_2018_01.sum())
# Few titles acquired during the pandemic, as you'd expect.
print(df_by_country.TITLES_2021_08.sum() - df_by_country.TITLES_2020_01.sum())

32232
32261
31697
31569
28076
3371


We want to normalise this by the total number of titles. There were [25,945,597 registered titles in 2021](https://www.gov.uk/government/publications/hm-land-registry-annual-report-and-accounts-2020-to-2021) and ["22.8 million" registered titles in 2010](https://www.gov.uk/government/publications/land-registry-annual-report-and-accounts-2010-to-2011). This tells us that the percentage of all titles registered to overseas individuals has gone from 0.39% to 0.95% over the period.

In [521]:
print("{:.2f}%".format(df_by_country.TITLES_2010_01.sum() / 22800000 * 100.0), "of titles in 2010")
print("{:.2f}%".format(df_by_country.TITLES_2021_08.sum() / 25945597 * 100.0), "of titles in 2021")

0.39% of titles in 2010
0.95% of titles in 2021


Overseas companies [owned](https://www.gov.uk/guidance/hm-land-registry-overseas-companies-that-own-property-in-england-and-wales#access-the-data) 94,712 titles in August 2021, so altogether, overseas companies and individuals owned 1.32% of titles in 2021.

In [562]:
print("{:.2f}%".format((df_by_country.TITLES_2021_08.sum() + 94712) / 25945597 * 100.0), "of titles in 2021")

1.32% of titles in 2021


There are 214 countries represented (the data includes some things that aren't really countries, like the Crown dependencies and overseas territories).

In [448]:
len(df_by_country)

214

The top 10 countries account for more than 50% of total titles owned overseas, and the top 20 for more than 75%.

In [11]:
print(df_by_country.sort_values("TITLES_2021_08", ascending=False).head(10).TITLES_2021_08.sum())
print(df_by_country.sort_values("TITLES_2021_08", ascending=False).head(20).TITLES_2021_08.sum())
print(df_by_country.TITLES_2021_08.sum())

135956
193968
247016


## Countries with most owners

The top 20 countries make up 75% of the total, so let's have a look at them here. These fall into a few distinct groups:

- South East Asia: Hong Kong, Singapore, Malaysia, China, Taiwan - lowish numbers in 2010, bigger rises over the period than other group
- Crown dependencies and British overseas territories (i.e. tax havens): Jersey, Guernsey, Isle of Man, British Virgin Islands - healthy numbers already by 2010, generally bigger rises earlier in the decade, still rising but slowing now.
- Countries with historic links to the UK: the US, Australia, South Africa, France - quite stong rises. Just people buying second homes?
- The Middle East: UAE, Saudi Arabia, Qatar - also big rises
- Ireland (combined with Eire at data processing stage) - outlier in having a big drop over the period. People having to sell up because of the financial crisis? Or an ageing population? I'd like to understand this better. (NB there aren't big drops anywhere else, though there is a small fall in North Korea. "Channel Islands" rises and falls over the period, though that may just reflect addressing conventions given the big rises in Jersey and Guernsey?)

In [450]:
df_by_country.sort_values("TITLES_2021_08", ascending=False).head(20)

,COUNTRY,TITLES_2010_01,TITLES_2012_01,TITLES_2014_01,TITLES_2016_01,TITLES_2018_01,TITLES_2020_01,TITLES_2021_08,abs_change_10_21,percent_change_10_21
0,HONG KONG,2170,4300,7278,11731,17382,21671,23524,21354,984.055300
1,JERSEY,6125,9062,11983,15791,16977,18293,18493,12368,201.926531
2,SINGAPORE,7323,9928,12735,15016,16822,18173,18165,10842,148.054076
3,UNITED STATES OF AMERICA,5292,6622,7922,9753,11152,13032,13076,7784,147.089947
4,IRELAND,18759,18743,17075,14781,13242,12268,12056,-6703,-35.732182
5,GUERNSEY,3783,6584,8550,10045,10874,11495,11594,7811,206.476342
6,UNITED ARAB EMIRATES,2288,3139,4478,6080,8512,10731,10914,8626,377.010490
7,MALAYSIA,1987,3886,5302,6952,9621,10129,10177,8190,412.179165
8,ISLE OF MAN,5334,7358,8593,8786,9331,9372,9338,4004,75.065617
9,AUSTRALIA,2848,4171,5461,6353,7415,8639,8619,5771,202.633427


For each major group in the top 12, print the total now and the net increase since 2010.

In [21]:
groups = [
    ["HONG KONG", "SINGAPORE", "MALAYSIA"],
    ["JERSEY", "GUERNSEY", "ISLE OF MAN", "BRITISH VIRGIN ISLANDS"],
    ["UNITED STATES OF AMERICA", "IRELAND", "AUSTRALIA"],
    ["UNITED ARAB EMIRATES", "KUWAIT"],
]
for group in groups:
    print("%s:" % group)
    print(df_by_country[df_by_country.COUNTRY.isin(group)].TITLES_2021_08.sum())
    print(df_by_country[df_by_country.COUNTRY.isin(group)].TITLES_2021_08.sum()-df_by_country[df_by_country.COUNTRY.isin(group)].TITLES_2010_01.sum())

['HONG KONG', 'SINGAPORE', 'MALAYSIA']:
51866
40386
['JERSEY', 'GUERNSEY', 'ISLE OF MAN', 'BRITISH VIRGIN ISLANDS']:
47323
29527
['UNITED STATES OF AMERICA', 'IRELAND', 'AUSTRALIA']:
33751
6852
['UNITED ARAB EMIRATES', 'KUWAIT']:
18869
15099


Check out the increase between 2010 and 2021. Which countries contributed most to that?

The absolute number of titles owned overseas rose by 160k over the period.

In [481]:
df_by_country.abs_change_10_21.sum()

159206

Just 12 countries were responsible for more than two-thirds of the increase.

In [495]:
df_by_country.sort_values("abs_change_10_21", ascending=False).head(12).abs_change_10_21.sum()

106998

There were particularly large and steep rises in:
    
- Hong Kong, Singapore, Malaysia and China, with 47,213 new titles - the number of Hong Kong titles rose 10 times, China titles rose 25 times
- UAE, Kuwait, and Saudi Arabia, with 20,707 new titles - all up over 4 times

And also big rises (from a higher original base) in: 

- Jersey, Guernsey, and British Virgin Islands, with 25,523 new titles - all up roughly 3 times
- The US and Australia, with 13,555 new titles - up 2-3 times

In [497]:
df_by_country.sort_values("abs_change_10_21", ascending=False)\
    [['COUNTRY', 'TITLES_2010_01', 'TITLES_2021_08', 'abs_change_10_21', 'percent_change_10_21']].head(12)

,COUNTRY,TITLES_2010_01,TITLES_2021_08,abs_change_10_21,percent_change_10_21
0,HONG KONG,2170,23524,21354,984.055300
1,JERSEY,6125,18493,12368,201.926531
2,SINGAPORE,7323,18165,10842,148.054076
6,UNITED ARAB EMIRATES,2288,10914,8626,377.010490
7,MALAYSIA,1987,10177,8190,412.179165
5,GUERNSEY,3783,11594,7811,206.476342
3,UNITED STATES OF AMERICA,5292,13076,7784,147.089947
13,CHINA,251,7078,6827,2719.920319
10,KUWAIT,1482,7955,6473,436.774629
9,AUSTRALIA,2848,8619,5771,202.633427


Check out the countries with the biggest increases, relative to their starting point.

In [501]:
df_by_country[df_by_country.TITLES_2021_08 > 500].sort_values("percent_change_10_21", ascending=False).head()

,COUNTRY,TITLES_2010_01,TITLES_2012_01,TITLES_2014_01,TITLES_2016_01,TITLES_2018_01,TITLES_2020_01,TITLES_2021_08,abs_change_10_21,percent_change_10_21
33,TAIWAN,29,59,100,431,1005,1199,1230,1201,4141.379310
13,CHINA,251,463,896,1793,3451,6487,7078,6827,2719.920319
35,RUSSIAN FEDERATION,86,179,404,710,985,1130,1127,1041,1210.465116
0,HONG KONG,2170,4300,7278,11731,17382,21671,23524,21354,984.055300
18,QATAR,302,483,1400,1852,2475,3135,3189,2887,955.960265


Finally, the number of titles fell over the period in Ireland and North Korea.

In [479]:
df_by_country[df_by_country.TITLES_2021_08 > 0].sort_values("abs_change_10_21").head(5)

,COUNTRY,TITLES_2010_01,TITLES_2012_01,TITLES_2014_01,TITLES_2016_01,TITLES_2018_01,TITLES_2020_01,TITLES_2021_08,abs_change_10_21,percent_change_10_21
4,IRELAND,18759,18743,17075,14781,13242,12268,12056,-6703,-35.732182
132,THE PEOPLES REPUBLIC OF KOREA,61,56,40,32,23,16,15,-46,-75.409836
87,ICELAND,99,94,86,89,87,91,90,-9,-9.090909
124,MARSHALL ISLANDS,27,37,31,39,30,22,21,-6,-22.222222
203,MONTSERRAT,4,4,3,3,2,1,1,-3,-75.000000


## By district

Ideally we might normalise this by the total titles per district, but this isn't something Land Registry publish, I think (there are stats on the [total dwelling stock](https://www.gov.uk/government/statistical-data-sets/live-tables-on-dwelling-stock-including-vacants) per district - see Table 116 - but dwelling stock of course isn't the same as titles).

In [561]:
df_dist = pd.read_csv("by_dist.csv")
df_dist.head(50)

,DIST,TITLES_2010_01,TITLES_2012_01,TITLES_2014_01,TITLES_2016_01,TITLES_2018_01,TITLES_2020_01,TITLES_2021_08,abs_change_10_21,percent_change_10_21
0,CITY OF WESTMINSTER,3584,5693,8713,10438,10856,11948,12104,8520,237.723214
1,LIVERPOOL,1964,2311,3272,4355,6029,7883,7925,5961,303.513238
2,TOWER HAMLETS,1878,3490,4452,4968,6142,7287,7547,5669,301.863685
3,KENSINGTON AND CHELSEA,2482,4060,5131,5960,6594,7077,7073,4591,184.971797
4,BUCKINGHAMSHIRE,1847,3696,4842,6155,6299,6599,6609,4762,257.823498
5,SALFORD,1681,2125,2264,2702,3720,5069,5294,3613,214.931588
6,WANDSWORTH,1309,1994,2456,3131,4246,5031,5110,3801,290.374332
7,HOUNSLOW,4133,4251,4480,4612,4784,4910,4923,790,19.114445
8,MANCHESTER,2484,2807,2887,3287,4104,4376,4793,2309,92.954911
9,SOUTHWARK,747,1202,1626,2084,3064,3772,3796,3049,408.165997


In [462]:
len(df_dist)

333

In [564]:
london_boroughs = ['BARKING AND DAGENHAM', 'BARNET',  'BEXLEY', 'BRENT', 
                   'BROMLEY', 'CAMDEN', 'CITY OF LONDON', 'CROYDON', 'EALING', 
                  'ENFIELD', 'GREENWICH', 'HACKNEY', 'HAMMERSMITH AND FULHAM',
                  'HARINGAY', 'HARROW', 'HARINGEY', 'HAVERING', 'HILLINGDON', 'HOUNSLOW', 
                  'ISLINGTON', 'KENSINGTON AND CHELSEA', 'KINGSTON UPON THAMES', 
                  'LAMBETH', 'LEWISHAM', 'MERTON', 'NEWHAM', 'REDBRIDGE', 
                  'RICHMOND UPON THAMES', 'SOUTHWARK', 'SUTTON', 'TOWER HAMLETS', 
                  'WALTHAM FOREST', 'WANDSWORTH', 'CITY OF WESTMINSTER']
df_london = df_dist[df_dist.DIST.isin(london_boroughs)]
df_london

,DIST,TITLES_2010_01,TITLES_2012_01,TITLES_2014_01,TITLES_2016_01,TITLES_2018_01,TITLES_2020_01,TITLES_2021_08,abs_change_10_21,percent_change_10_21
0,CITY OF WESTMINSTER,3584,5693,8713,10438,10856,11948,12104,8520,237.723214
2,TOWER HAMLETS,1878,3490,4452,4968,6142,7287,7547,5669,301.863685
3,KENSINGTON AND CHELSEA,2482,4060,5131,5960,6594,7077,7073,4591,184.971797
6,WANDSWORTH,1309,1994,2456,3131,4246,5031,5110,3801,290.374332
7,HOUNSLOW,4133,4251,4480,4612,4784,4910,4923,790,19.114445
9,SOUTHWARK,747,1202,1626,2084,3064,3772,3796,3049,408.165997
11,CAMDEN,1013,1556,2223,2627,3195,3621,3666,2653,261.895360
12,HAMMERSMITH AND FULHAM,748,1205,1667,2133,2767,3472,3533,2785,372.326203
13,LAMBETH,816,1916,2107,2389,2922,3374,3483,2667,326.838235
14,NEWHAM,598,735,808,1228,2036,2949,3045,2447,409.197324


London makes up about a third of the total, and about a third of the rise of the period.

In [571]:
df_london['DIST_TITLE'] = df_london.DIST.str.title()
df_london.DIST_TITLE.replace({"City Of Westminster": "Westminster"}, inplace=True)
df_london[["DIST_TITLE", "TITLES_2021_08", "abs_change_10_21"]].to_csv('london.csv', index=False)

In [572]:
print(df_london.TITLES_2021_08.sum())
print(df_london.abs_change_10_21.sum())

85451
58981


Other major city centres account for about 31,000 titles. 

In [558]:
city_centres = ['LIVERPOOL', 'SEFTON', 'MANCHESTER', 'SALFORD', 'TRAFFORD',
                'LEEDS', 'BIRMINGHAM', 'SHEFFIELD', 'CITY OF NOTTINGHAM']
df_cities = df_dist[df_dist.DIST.isin(city_centres)]
print(df_cities.TITLES_2021_08.sum())
print(df_cities.abs_change_10_21.sum())

31520
20917


Which districts have seen the most dramatic rises?

London dominates but there are also city centres outside London, with Liverpool, Salford, Birmingham, Manchester in here.

There are also some Home Counties boroughs, in Buckinghamshire, Tandridge (in east Surrey), North Northamptonshire and Central Bedfordshire. Wonder what the latter arae about?

Just outside the top 20 we also have Leeds, Sefton, Sheffield, Trafford and Nottingham. 

We also have Cornwall, Bournemouth, and Brighton in the top 40. 

In [555]:
df_dist.sort_values("abs_change_10_21", ascending=False).head(40)

,DIST,TITLES_2010_01,TITLES_2012_01,TITLES_2014_01,TITLES_2016_01,TITLES_2018_01,TITLES_2020_01,TITLES_2021_08,abs_change_10_21,percent_change_10_21
0,CITY OF WESTMINSTER,3584,5693,8713,10438,10856,11948,12104,8520,237.723214
1,LIVERPOOL,1964,2311,3272,4355,6029,7883,7925,5961,303.513238
2,TOWER HAMLETS,1878,3490,4452,4968,6142,7287,7547,5669,301.863685
4,BUCKINGHAMSHIRE,1847,3696,4842,6155,6299,6599,6609,4762,257.823498
3,KENSINGTON AND CHELSEA,2482,4060,5131,5960,6594,7077,7073,4591,184.971797
6,WANDSWORTH,1309,1994,2456,3131,4246,5031,5110,3801,290.374332
5,SALFORD,1681,2125,2264,2702,3720,5069,5294,3613,214.931588
9,SOUTHWARK,747,1202,1626,2084,3064,3772,3796,3049,408.165997
15,TANDRIDGE,201,603,2298,2411,2950,3019,3014,2813,1399.502488
12,HAMMERSMITH AND FULHAM,748,1205,1667,2133,2767,3472,3533,2785,372.326203


In [534]:
df_dist.sort_values("TITLES_2021_08", ascending=False).head(15).TITLES_2021_08.sum()

82599

In [530]:
These rises are less clustered than 

159206

In [533]:
df_dist.sort_values("abs_change_10_21", ascending=False).head(15).abs_change_10_21.sum()

58303

Now we want to dive into what's going on in those districts. 

In [525]:
top_districts = df_dist.sort_values("abs_change_10_21", ascending=False).DIST.head(10)
top_districts

0        CITY OF WESTMINSTER
1                  LIVERPOOL
2              TOWER HAMLETS
4            BUCKINGHAMSHIRE
3     KENSINGTON AND CHELSEA
6                 WANDSWORTH
5                    SALFORD
9                  SOUTHWARK
15                 TANDRIDGE
12    HAMMERSMITH AND FULHAM
Name: DIST, dtype: object

## By district and local authority

The files are sorted by the absolute change between 2010 and 2021. Where are the hotspots?

- This tells us that there's always been a large number of Singapore-owned titles in Hounslow
- It also shows sudden bursts of activity in particular districts e.g. Kuwaiti owners in Buckinghamshire?! - is that a particular development, or marketing push?

In [456]:
df_dc = pd.read_csv("df_by_country_and_dist.csv")
df_dc.head(10)

,COUNTRY,DIST,TITLES_2010_01,TITLES_2012_01,TITLES_2014_01,TITLES_2016_01,TITLES_2018_01,TITLES_2020_01,TITLES_2021_08,abs_change_10_21,percent_change_10_21
0,SINGAPORE,HOUNSLOW,3685,3665,3669,3684,3691,3692,3692,7,0.189959
1,KUWAIT,BUCKINGHAMSHIRE,555,578,1099,2108,2168,2219,2219,1664,299.819820
2,HONG KONG,TOWER HAMLETS,185,689,1020,1183,1544,1846,1959,1774,958.918919
3,JERSEY,CITY OF WESTMINSTER,518,827,1919,2306,1505,1545,1564,1046,201.930502
4,HONG KONG,SALFORD,33,55,98,392,879,1361,1562,1529,4633.333333
5,MALAYSIA,NORTH NORTHAMPTONSHIRE,0,1,3,95,1062,1455,1465,1465,inf
6,HONG KONG,MANCHESTER,45,58,76,327,954,1053,1270,1225,2722.222222
7,HONG KONG,CITY OF WESTMINSTER,132,254,455,680,952,1205,1232,1100,833.333333
8,BRITISH VIRGIN ISLANDS,CITY OF WESTMINSTER,443,817,1023,1193,1208,1193,1190,747,168.623025
9,HONG KONG,LIVERPOOL,14,36,175,432,815,1066,1097,1083,7735.714286


This lets us dive into the patterns for particular countries, or districts, a bit more. 

In [459]:
df_dc[df_dc.COUNTRY == "HONG KONG"].head(10)

,COUNTRY,DIST,TITLES_2010_01,TITLES_2012_01,TITLES_2014_01,TITLES_2016_01,TITLES_2018_01,TITLES_2020_01,TITLES_2021_08,abs_change_10_21,percent_change_10_21
2,HONG KONG,TOWER HAMLETS,185,689,1020,1183,1544,1846,1959,1774,958.918919
4,HONG KONG,SALFORD,33,55,98,392,879,1361,1562,1529,4633.333333
6,HONG KONG,MANCHESTER,45,58,76,327,954,1053,1270,1225,2722.222222
7,HONG KONG,CITY OF WESTMINSTER,132,254,455,680,952,1205,1232,1100,833.333333
9,HONG KONG,LIVERPOOL,14,36,175,432,815,1066,1097,1083,7735.714286
16,HONG KONG,SOUTHWARK,47,149,220,338,502,780,793,746,1587.234043
20,HONG KONG,NEWHAM,10,23,62,203,485,691,741,731,7310.000000
21,HONG KONG,BIRMINGHAM,20,37,105,158,253,628,728,708,3540.000000
31,HONG KONG,CAMDEN,30,99,258,320,476,558,570,540,1800.000000
32,HONG KONG,KENSINGTON AND CHELSEA,109,192,284,383,475,545,564,455,417.431193


In [460]:
df_dc[df_dc.DIST == "SALFORD"].head(10)

,COUNTRY,DIST,TITLES_2010_01,TITLES_2012_01,TITLES_2014_01,TITLES_2016_01,TITLES_2018_01,TITLES_2020_01,TITLES_2021_08,abs_change_10_21,percent_change_10_21
4,HONG KONG,SALFORD,33,55,98,392,879,1361,1562,1529,4633.333333
42,IRELAND,SALFORD,1075,1064,953,770,608,497,487,-588,-54.697674
50,BRITISH VIRGIN ISLANDS,SALFORD,3,4,25,128,83,450,452,449,14966.666667
97,CHINA,SALFORD,2,2,8,44,132,268,302,300,15000.000000
105,UNITED ARAB EMIRATES,SALFORD,26,30,41,82,226,281,289,263,1011.538462
116,SINGAPORE,SALFORD,5,11,100,147,179,289,273,268,5360.000000
153,ISLE OF MAN,SALFORD,382,641,549,416,295,227,224,-158,-41.361257
180,KUWAIT,SALFORD,3,2,4,32,126,178,181,178,5933.333333
209,SAUDI ARABIA,SALFORD,1,1,7,21,102,152,154,153,15300.000000
358,GUERNSEY,SALFORD,10,96,112,110,98,100,101,91,910.000000


In [461]:
df_dc[df_dc.DIST == "MANCHESTER"].head(10)

,COUNTRY,DIST,TITLES_2010_01,TITLES_2012_01,TITLES_2014_01,TITLES_2016_01,TITLES_2018_01,TITLES_2020_01,TITLES_2021_08,abs_change_10_21,percent_change_10_21
6,HONG KONG,MANCHESTER,45,58,76,327,954,1053,1270,1225,2722.222222
24,IRELAND,MANCHESTER,1649,1581,1368,1062,785,657,651,-998,-60.521528
119,JERSEY,MANCHESTER,61,75,77,155,226,269,270,209,342.622951
120,GUERNSEY,MANCHESTER,110,123,197,224,171,194,269,159,144.545455
142,SINGAPORE,MANCHESTER,9,37,51,98,126,222,238,229,2544.444444
149,BRITISH VIRGIN ISLANDS,MANCHESTER,10,17,27,82,302,201,231,221,2210.000000
158,UNITED ARAB EMIRATES,MANCHESTER,64,96,108,119,172,200,208,144,225.000000
170,CHINA,MANCHESTER,2,4,21,61,112,168,191,189,9450.000000
253,ISLE OF MAN,MANCHESTER,219,357,299,227,143,130,130,-89,-40.639269
387,UNITED STATES OF AMERICA,MANCHESTER,31,42,55,62,77,95,96,65,209.677419


In [468]:
df_dc[['COUNTRY', 'DIST', 'TITLES_2021_08']].to_csv("joint_heatmap.csv")

In [ ]:
We can look at what percentage of transactions in particular districts were to overseas investors. 